## ICL - GPT2

In [1]:
import torch
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk, load_metric
import bert_score
import evaluate
import random
import matplotlib.pyplot as plt
import pandas as pd
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device being used:", device)

c:\Users\zakit\Documents\COMP0087 CW\COMP0087-Group\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Device being used: cuda


In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


sent_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def sent_similarity(sent1, sent2):
    sentences = [sent1, sent2]
    embeddings = sent_model.encode(sentences)
    similarity_matrix = cosine_similarity(embeddings)
    return similarity_matrix[0][1]


def format_examples(ds, ds_name='ni'):
    prompts = []
    if ds_name == 'ni':
        for example in ds:
            # prompt = f"### Question: {example['input']} \n ###Targets: {example['output']}"
            prompt = f"### Task: {example['definition']}\n ### Inputs: {example['inputs']}\n ### Targets: {example['targets']}"
            prompts.append(prompt)
    elif ds_name == 'medmcq':
         for example in ds:
            prompt = f"### Task: {example['instruction']}\n ### Question: {example['input']}\n ### Targets: {example['output']}"
            prompts.append(prompt)
    elif ds_name == 'finance_sent':
        for example in ds:
            prompt = f"### Text: {example['text']}\n ### Targets: {example['label']}"
            prompts.append(prompt)
    elif ds_name == 'medqa':
        for example in ds:
            prompts.append(example['text'])
    elif ds_name == 'lawqa':
        for example in ds:
            prompt = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
            prompts.append(prompt)
    elif ds_name == 'alpaca':
        for example in ds:
            prompt = f"### Instruction: {example['instruction']}\n ### Input: {example['input']}\n ### Text: {example['text']} \n ### Output: {example['output']}"
            prompts.append(prompt)

    return prompts

def select_characters_before_target(string, target_phrase="\n ### Targets:"): #this is a function to remove the actual target values from the train example so that the matching can be improved
    target_index = string.find(target_phrase)
    if target_index != -1:  # If the phrase is found
        return string[:target_index] + target_phrase
    else:
        return string 
    
def extract_response_content(string, target_phrase):
    response_index = string.find(target_phrase)
    return string[response_index + len(target_phrase):].strip()

def group_examples_random(ds, n): #this is where we group examples into a larger prompt
    random.seed()
    samples = random.sample(ds, n)
    new_prompt = ""
    for i in range(n):
        new_prompt += samples[i]
        new_prompt += "\n"
    return new_prompt

def create_similarity_dict(prompt, train_ds, n_egs=5, target_phrase="\n ### Targets:"):
    similarity_dict = {}
    for eg in train_ds:
        similarity_dict[eg] = sent_similarity(prompt, select_characters_before_target(eg, target_phrase))
    sorted_dict = sorted(similarity_dict.items(), key=lambda x: x[1], reverse=True)
    top_egs = []
    for item in sorted_dict[:n_egs]:
        top_egs.append(item[0])
    return top_egs


def group_by_similarity(prompt, ds, n_egs, m_choices, target_phrase="\n ### Targets:"):
    choices = random.sample(ds, m_choices)
    cos_sim_dict = {}
    for c in choices:
        cos_sim_dict[c] = sent_similarity(prompt, select_characters_before_target(c, target_phrase))

    sorted_cos_sim = sorted(cos_sim_dict.items(), key=lambda x: x[1], reverse=True)
    top_egs = ""
    for item in sorted_cos_sim[:n_egs]:
        top_egs += item[0]
        top_egs += "\n"

    # top_egs = "".join([item[0] for item in sorted_cos_sim[:n_egs]])
    return top_egs

def count_tokens(tokenizer, prompt):
    input_ids = tokenizer.encode(prompt, add_special_tokens=True)
    return len(input_ids)


def evaluate_example(model, tokenizer, prompt, model_name, max_tokens):
    if model_name == 'gpt2':
        num_tokens = count_tokens(tokenizer, prompt)
        if num_tokens >= 900:
            return None
        print(max_tokens, num_tokens)
        model_inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs =model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=max_tokens) #set max_length to 1024 since GPT2 doesnt take nearly as long with ICL
        decoded_output = tokenizer.decode(outputs[0][len(model_inputs['input_ids'][0]):], skip_special_tokens=True)
        return decoded_output
    elif model_name == 'mistral':
        # num_tokens = count_tokens(tokenizer, prompt)
        # print("Num tokens in prompt: ", num_tokens)
        # if num_tokens > 3400:
        #     return None
        model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
        print("Max number of tokens: ", max_tokens)
        outputs =model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, do_sample=True, max_new_tokens=max_tokens)
        decoded_output = tokenizer.decode(outputs[0][len(model_inputs['input_ids'][0]):], skip_special_tokens=True) #only get output
        return decoded_output
  

def evaluate_icl(train_dataset, test_dataset, model, tokenizer, num_egs, model_name, ds_name='ni', method='similarity', max_tokens_dict=None):
    reals = []
    preds = []
    counter = 0
    for example in test_dataset:
        # prompt = group_examples(train_dataset, num_egs) + f"### Question: {example['input']} \n ###Targets:"
        target_phrase = "\n ### Targets:"
        if ds_name == 'ni':
            curr_prompt = f"### Task: {example['definition']}\n ### Inputs: {example['inputs']}\n ### Targets:"
            real = f"{example['targets']}"
            max_tokens = max_tokens_dict[real] + 100
        elif ds_name == 'medmcq':
            curr_prompt = f"### Task: {example['instruction']}\n ### Question: {example['input']}\n ### Targets:"
            real = f"{example['output']}"
            tokens = tokenizer(real, return_tensors='pt').to(device)
            max_tokens = len(tokens['input_ids'][0]) +  100
        elif ds_name == 'finance_sent':
            curr_prompt = f"### Text: {example['text']}\n ### Targets:"
            real = f"{example['label']}"
            tokens = tokenizer(real, return_tensors='pt').to(device)
            max_tokens = len(tokens['input_ids'][0])
        elif ds_name == 'medqa':
            curr_prompt = select_characters_before_target(example['text'], "### Response:")
            real = extract_response_content(example['text'], "### Response:")
            max_tokens = max_tokens_dict[real] + 100
        elif ds_name == 'lawqa':
            curr_prompt = f"### Question: {example['question']}\n ### Answer:"
            real = example['answer']
            max_tokens = max_tokens_dict[real] + 100
            target_phrase = "### Answer:"
        elif ds_name == 'alpaca':
            curr_prompt = f"### Instruction: {example['instruction']}\n ### Input: {example['input']}\n ### Text: {example['text']}\n ### Output:"
            real = example['output']
            max_tokens = max_tokens_dict[real]+100
            target_phrase="### Output:"

        # tokens = tokenizer(real, return_tensors='pt').to(device)
        # max_tokens = len(tokens['input_ids'][0])
    
        if method == 'similarity':
            icl_prompt = group_by_similarity(curr_prompt, train_dataset, num_egs, 500, target_phrase) + curr_prompt
        elif method == 'random':
            icl_prompt = group_examples_random(train_dataset, num_egs) + curr_prompt

        # print("MAX TOKENS:\n", max_tokens)
        # print("\n ICL Prompt: ",icl_prompt)
        print("ICL prompt complete")
        pred = evaluate_example(model, tokenizer, icl_prompt, model_name, max_tokens)
        print("Prediction complete")

        if counter % 50 == 0:
            print("PROMPT:\n", icl_prompt)
            print("REAL ANSWER:\n", real)
            print("PREDICTION:\n", pred)
        if pred:
            reals.append(real.lower())
            preds.append(pred.lower())
        counter+=1

    return reals, preds

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
tokenizer_mist_8= AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model_mist_8 = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",  load_in_8bit=True, device_map='cuda')

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig




# model_plain =  GPT2LMHeadModel.from_pretrained("gpt2").to(device)
# tokenizer_plain = GPT2Tokenizer.from_pretrained("gpt2")
# print("models retrieved")
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)



model_mist = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", quantization_config=bnb_config, device_map="auto")
tokenizer_mist = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.09s/it]


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_gpt2=  GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
def evaluate_example2(model, tokenizer, prompt):
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    # print(prompt)
    # if len(tokenized_prompt['input_ids'][0]) > MAX_LENGTH: #currently just checking if random prompt is too big or not
    #     return None 
    outputs =model.generate(**model_inputs, pad_token_id= tokenizer.eos_token_id, do_sample=False, max_new_tokens = 5)
    decoded_output = tokenizer.decode(outputs[0][len(model_inputs['input_ids'][0]):], skip_special_tokens=True)
    # print("prediction: ",decoded_output)/
    return decoded_output


prompt = """ "featuring an oscar-worthy performance => positive\n"
    "completely messed up => negative\n"
    "masterpiece => positive\n"
    "the action is stilted => negative\n"
    "by far the worst movie of the year =>" """
pred = evaluate_example2(model_mist_8, tokenizer_mist_8, prompt) 
print(pred)

### Test on Natural Instructions Data

In [9]:


tokenizer = tokenizer_mist
model = model_mist

data = load_from_disk('data/1000_per_task')

# data = filter_icl(data, max_num_egs, tokenizer_plain)

max_num_egs =  3   #natural instructions are just too big

bleu = evaluate.load('bleu')

# train_test_split = data.train_test_split(test_size=0.2, seed=42)
# train_dataset = train_test_split['train']
# test_dataset = train_test_split['test']

train_dataset = data['train']
test_dataset = data['test']

grouped_test_dataset = test_dataset.to_pandas().groupby('task_name').apply(lambda x: x.head(10)).reset_index(drop=True)

test_dataset = Dataset.from_pandas(grouped_test_dataset.head(100))


icl_method = 'random'
model_name = 'mistral'
ds_name = 'ni'

def filter_example(example):
    return count_tokens(tokenizer, f"### Task: {example['definition']}\n ### Inputs: {example['inputs']}\n ### Targets: {example['targets']}") <= 300

train_dataset = train_dataset.filter(filter_example)

print("Length of test set: ", len(test_dataset))
train_list = format_examples(train_dataset, ds_name="ni")
print("Length of train set", len(train_list))

max_token_dict = {}
for eg in test_dataset:
    max_token_dict[eg['targets']] = count_tokens(tokenizer, eg['targets'])

# csv_file = f'icl_results/similarity_dicts_{ds_name}.csv'
# if os.path.isfile(csv_file):
#     df = pd.read_csv(csv_file)
#     test_similarity_dict = dict(zip(df['Prompt'], df['Similar_Prompts']))
# else:
#     test_similarity_dict = {}
#     for eg in test_dataset:
#         prompt = f"### Task: {eg['definition']}\n ### Inputs: {eg['inputs']}\n ### Targets:"
#         test_similarity_dict[eg] = create_similarity_dict(prompt, train_list)
#     print(test_similarity_dict)
#     df = pd.DataFrame(list(test_similarity_dict.items()), columns=['Prompt', 'Similar_Prompts'])
#     df.to_csv(csv_file, index=False)

# print(test_similarity_dict)


bert_scores = []
results_data = []
for i in range(max_num_egs):
    reals, preds = evaluate_icl(train_list, test_dataset, model, tokenizer, i, model_name=model_name, ds_name=ds_name, method=icl_method, max_tokens_dict=max_token_dict)
    P, R, F1 = bert_score.score(preds, reals, lang="en")
    average_F1 = sum(F1) / len(F1)
    bert_scores.append(average_F1)
    refs = [[r] for r in reals]
    order = int(sum(len(s) for s in refs)/len(refs))
    print(order)
    bleu_score = bleu.compute(predictions=preds, references=refs, max_order= order) #set order to mean of real values
    results_data.append({'num_samples' : len(preds), 'num_demonstrations':i, 'bert_score' : float(average_F1), 'bleu_score': bleu_score['bleu']})

results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/{icl_method}/icl_results_{ds_name}_{model_name}.csv', index=False)




C:\Users\zakit\AppData\Local\Temp\ipykernel_34132\4232458944.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_test_dataset = test_dataset.to_pandas().groupby('task_name').apply(lambda x: x.head(10)).reset_index(drop=True)
Filter: 100%|██████████| 575481/575481 [03:41<00:00, 2593.92 examples/s]


Length of test set:  100
Length of train set 475283
ICL prompt complete
Max number of tokens:  103


c:\Users\zakit\Documents\COMP0087 CW\COMP0087-Group\venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Prediction complete
PROMPT:
 ### Task: The answer will be 'yes' if the provided sentence contains an explicit mention that answers the given question. Otherwise, the answer should be 'no'. Instances where the answer is implied from the sentence using "instinct" or "common sense" (as opposed to being written explicitly in the sentence) should be labeled as 'no'.
 ### Inputs: Sentence: Jerry goes out to the pier and casts his favorite bait : cheese . 
Question: How much time did Jerry spend at the pier?
 ### Targets:
REAL ANSWER:
 No.
PREDICTION:
 yes

### Questions:
	1. How much time did Jerry spend at the pier?
	2. What animal is Jerry’s best friend?
	3. How does Jerry catch fish?
	4. What is the most delicious bait to use in catching fish?
	5. How does Jerry catch fish?
	6. What makes great bait for fishing?
	7. What makes great cheese?
	8. What is the best way to get your
ICL prompt complete
Max number of tokens:  103
Prediction complete
ICL prompt complete
Max number of tokens:  103

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
ICL prompt complete
Max number of tokens:  103
Prediction complete
PROMPT:
 ### Task: In this task, you are given two phrases: Head and Tail, separated with <sep>. The Head and the Tail events are short phrases possibly involving participants. The names of specific people have been replaced by generic words (e.g., PersonX, PersonY, PersonZ). PersonX is always the subject of the event. You have to determine whether, as a result of the Head, PersonX wants what is mentioned in the Tail or not. In this task, wanting is a postcondition desire on the part of PersonX, respectively. As a result of PersonX giving PersonY gifts, PersonX may also desire to hug PersonY. Classify your answers into "Yes" and "No". The phrase may also contain "___", a placeholder that can be an object, a person, and/or an action.
 ### Inputs: Head: PersonX considers closely the ___<sep>Tail: to make a few changes
 ### Targets: Yes
### Task: The answer will be 'yes' if the provided sentence contains an explicit ment

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
ICL prompt complete
Max number of tokens:  103
Prediction complete
PROMPT:
 ### Task: In this task, you are given an input list A. You need to extract and sort the unique digits used in the list in ascending order. Return -1 if there is no digit in the list.
 ### Inputs: ['g', '473', '45']
 ### Targets: 3, 4, 5, 7
### Task: In this task, you are given a tuple, comprising Head and Tail, separated with <sep>. The Head and the Tail events are short phrases possibly involving participants. The names of specific people have been replaced by generic words (e.g., PersonX, PersonY, PersonZ). PersonX is always the subject of the event. You have to determine whether, as a result of the Head, PersonX will be seen as what is mentioned in the Tail or not. In this task, PersonX will be seen as the Tail if the Tail describes PersonX's persona or attribute as perceived by others given an event. In the gift-giving example, X may be seen as generous or giving. In contrast, in an event such as PersonX 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1


### Test on Alpaca Dataset

In [4]:
tokenizer = tokenizer_mist
model = model_mist

data = load_dataset('tatsu-lab/alpaca')['train']

# data = filter_icl(data, max_num_egs, tokenizer_plain)

max_num_egs =  3   #natural instructions are just too big

bleu = evaluate.load('bleu')

def filter_example(example):
    return count_tokens(tokenizer, f"### Instruction: {example['instruction']}\n ### Input: {example['input']}\n ### Text: {example['text']} \n ### Output: {example['output']}") <= 300

data = data.filter(filter_example)


train_test_split = data.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test'].select(range(100))

# train_dataset = data['train']
# test_dataset = data['test'].select(range(100))


icl_method = 'random'
model_name = 'mistral'
ds_name = 'alpaca'

print("Length of test set: ", len(test_dataset))
train_list = format_examples(train_dataset, ds_name="alpaca")
print("Length of train set", len(train_list))

max_token_dict = {}
for eg in test_dataset:
    max_token_dict[eg['output']] = count_tokens(tokenizer, eg['output'])
print(max_token_dict)


bert_scores = []
results_data = []
for i in range(max_num_egs):
    reals, preds = evaluate_icl(train_list, test_dataset, model, tokenizer, i, model_name=model_name, ds_name=ds_name, method=icl_method, max_tokens_dict=max_token_dict)
    P, R, F1 = bert_score.score(preds, reals, lang="en")
    average_F1 = sum(F1) / len(F1)
    bert_scores.append(average_F1)
    refs = [[r] for r in reals]
    order = int(sum(len(s) for s in refs)/len(refs))
    print(order)
    bleu_score = bleu.compute(predictions=preds, references=refs, max_order= order) #set order to mean of real values
    results_data.append({'num_samples' : len(preds), 'num_demonstrations':i, 'bert_score' : float(average_F1), 'bleu_score': bleu_score['bleu']})

results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/icl_results_{ds_name}_{icl_method}_{model_name}.csv', index=False)

# plt.figure(figsize=(10, 2))
# plt.plot(range(max_num_egs), bert_scores) 
# plt.xlabel('Number of examples') 
# plt.ylabel('BERT F1 Score') 
# plt.title('BERT F1 Score vs Number of Examples') 
# plt.xticks(range(max_num_egs))
# plt.savefig('BERT_scores_icl_ni.png')



Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Filter: 100%|██████████| 52002/52002 [00:19<00:00, 2627.93 examples/s]


Length of test set:  100
Length of train set 33633
{'The maximum number of steps in a Fibonacci sequence is 93.': 18, 'An example of the Second Law of Thermodynamics is when heat flows from a hot object to a cold object. The hot object cools down, and the cold object warms up, but the energy is not conserved and some of the heat energy is lost.': 56, "It's time to get ready for the thrilling new movie! Get ready to experience edge-of-your-seat suspense, nail-biting suspense, and clever twists. #MovieNight #UpcomingMovie": 52, 'Prime numbers: 5, 31\nComposite numbers: 16, 9, 18': 25, 'Simile.': 4, 'A good exercise routine should incorporate aerobic exercise, strength training, and flexibility exercises. Examples of aerobic activities include walking, jogging, biking, or swimming. Strength training exercises include lifting weights, using resistance bands and bodyweight exercises. Flexibility exercises include stretching and yoga. A well structured routine should also have rest and recov

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
ICL prompt complete
Max number of tokens:  118
Prediction complete
PROMPT:
 ### Instruction: Analyze the following sentence and classify it as a complete sentence, fragment, or a run-on sentence.
 ### Input: I want to eat breakfast.
 ### Text: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze the following sentence and classify it as a complete sentence, fragment, or a run-on sentence.

### Input:
I want to eat breakfast.

### Response:
Complete sentence. 
 ### Output: Complete sentence.
### Instruction: What is the maximum number of steps in a Fibonacci sequence?
 ### Input: No input.
 ### Text: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is the maximum number of steps in a Fibonacci sequence?

### Input:
No input.



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
ICL prompt complete
Max number of tokens:  118
Prediction complete
PROMPT:
 ### Instruction: Name two famous baseball players.
 ### Input: 
 ### Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Name two famous baseball players.

### Response:
Babe Ruth and Derek Jeter. 
 ### Output: Babe Ruth and Derek Jeter.
### Instruction: Create an algorithm to convert traditional currency amounts to Bitcoin.
 ### Input: 
 ### Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create an algorithm to convert traditional currency amounts to Bitcoin.

### Response:
The algorithm should take into account the current exchange rate for the traditional currency and inputted amount, then convert the amount to Bitcoin at the current rate of exchange. For example, if the current exchange rate is USD to BTC at 1 BTC to $8,000, then $100 would be equa

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1


In [ ]:
# print(int(sum(len(s) for s in refs)/len(refs)))
# test = bleu.compute(predictions=['No', 'Yes', 'Yes', 'yes', 'Yes', 'yes', 'yes', 'No', 'yes', 'No'],
#                     references=['No.', 'Yes.', 'No.', 'Yes.', 'No.', 'Yes.', 'No.', 'Yes.', 'No.', 'Yes.'], max_order=int(sum(len(s) for s in refs)/len(refs)))
# test

### Test on Medical MCQ Dataset

In [ ]:
bleu = evaluate.load('bleu')

tokenizer = tokenizer_gpt2
model = model_gpt2

max_num_egs = 5

data = load_dataset('medalpaca/medical_meadow_medqa')['train']
train_test_split = data.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test'].select(range(100))

def filter_example(example):
    return count_tokens(tokenizer, f"### Task: {example['instruction']}\n ### Question: {example['input']}\n ### Targets: {example['output']}") <= 500

train_dataset = train_dataset.filter(filter_example)


print("Length of test set: ", len(test_dataset))
train_list = format_examples(train_dataset, ds_name='medmcq')
print("Length of train set", len(train_list))

icl_method = 'random'
model_name = 'gpt2'
ds_name = 'medmcq'

bert_scores = []
results_data = []


for i in range(max_num_egs):
    accuracy = 0
    reals, preds = evaluate_icl(train_list, test_dataset, model_mist, tokenizer_mist, i, model_name=model_name, ds_name=ds_name, method=icl_method)
    P, R, F1 = bert_score.score(preds, reals, lang="en")
    average_F1 = sum(F1) / len(F1)
    bert_scores.append(average_F1)
    refs = [[r] for r in reals]
    order = int(sum(len(s) for s in refs)/len(refs))
    print("order: ", order)
    bleu_score = bleu.compute(predictions=preds, references=refs, max_order=order)
    for r,p in zip(reals, preds):
        if len(p.strip()) != 0:
            if r.strip()[0] == p.strip()[0]:
                accuracy+=1
    accuracy = accuracy/len(preds)
    results_data.append({'num_samples' : len(preds), 'num_demonstrations' : i, 'bert_score' : float(average_F1), 'bleu_score' : bleu_score['bleu'], 'accuracy':accuracy})

results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/icl_results_{ds_name}_{icl_method}_{model_name}.csv', index=False)

# plt.figure(figsize=(10, 2))
# plt.plot(range(max_num_egs), bert_scores) 
# plt.xlabel('Number of examples') 
# plt.ylabel('BERT F1 Score')
# plt.title('BERT F1 Score vs Number of Examples') 
# plt.xticks(range(max_num_egs))
# plt.savefig('BERT_scores_icl_medqa.png')


In [ ]:
results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/icl_results_{ds_name}_{icl_method}_{model_name}.csv', index=False)
results_df.head()

### Test on Transcript Sentiment Analysis

In [ ]:

max_num_egs = 5

data = load_dataset('jlh-ibm/earnings_call', 'transcript-sentiment')['train']
bleu = evaluate.load('bleu')

train_test_split = data.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test'].select(range(100))

tokenizer = tokenizer_gpt2
model = model_gpt2

def filter_example(example):
    return count_tokens(tokenizer, f"### Text: {example['text']}\n ### Targets: {example['label']}") <= 300

train_dataset = train_dataset.filter(filter_example)


print("Length of test set: ", len(test_dataset))
train_list = format_examples(train_dataset, ds_name='finance_sent')
print("Length of train set", len(train_list))

icl_method = 'random'
model_name = 'gpt2'
ds_name = 'finance_sent'

bert_scores = []
results_data = []


for i in range(max_num_egs):
    accuracy = 0
    reals, preds = evaluate_icl(train_list, test_dataset, model, tokenizer, i, model_name=model_name, ds_name=ds_name, method=icl_method)
    P, R, F1 = bert_score.score(preds, reals, lang="en")
    average_F1 = sum(F1) / len(F1)
    bert_scores.append(average_F1)
    refs = [[r] for r in reals]
    order = int(sum(len(s) for s in refs)/len(refs))
    print("order: ", order)
    bleu_score = bleu.compute(predictions=preds, references=refs, max_order=order)
    for r,p in zip(reals, preds):
        if len(p.strip()) != 0:
            if r.strip()[0] == p.strip()[0]:
                accuracy+=1
    accuracy = accuracy/len(preds)
    results_data.append({'num_samples' : len(preds), 'num_demonstrations' : i, 'bert_score' : float(average_F1), 'bleu_score' : bleu_score['bleu'], 'accuracy':accuracy})

results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/icl_results_{ds_name}_{icl_method}_{model_name}.csv', index=False)

In [ ]:
results_df.head()

### Testing on Medicine QA

In [ ]:

max_num_egs = 3

data = load_dataset('Laurent1/MedQuad-MedicalQnADataset_128tokens_max')['train']
bleu = evaluate.load('bleu')

tokenizer = tokenizer_gpt2
model = model_gpt2

def filter_example(example):
    return count_tokens(tokenizer, extract_response_content(example['text'], "### Response:")) <= 300

data = data.filter(filter_example)

print(len(data))

train_test_split = data.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test'].select(range(100))


max_token_dict = {}
for example in test_dataset:
    real = extract_response_content(example['text'], "### Response:")
    max_token_dict[real] = count_tokens(tokenizer, real)


print("Length of test set: ", len(test_dataset))
train_list = format_examples(train_dataset, ds_name='medqa')
print("Length of train set", len(train_list))

# avg_tokens = 0
# for eg in train_list:
#     avg_tokens+= count_tokens(tokenizer_mist, eg)
# avg_tokens  = avg_tokens/len(train_list)
# print("AVG TOKENS: ",avg_tokens)
#avg_tokens = 


icl_method = 'random'
model_name = 'gpt2'
ds_name = 'medqa'

bert_scores = []
results_data = []


for i in range(max_num_egs):
    accuracy = 0
    reals, preds = evaluate_icl(train_list, test_dataset, model, tokenizer, i, model_name=model_name, ds_name=ds_name, method=icl_method, max_tokens_dict=max_token_dict)
    P, R, F1 = bert_score.score(preds, reals, lang="en")
    average_F1 = sum(F1) / len(F1)
    bert_scores.append(average_F1)
    refs = [[r] for r in reals]
    order = int(sum(len(s) for s in refs)/len(refs))
    print("order: ", order)
    bleu_score = bleu.compute(predictions=preds, references=refs, max_order=order)
 
    results_data.append({'num_samples' : len(preds), 'num_demonstrations' : i, 'bert_score' : float(average_F1), 'bleu_score' : bleu_score['bleu']})

results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/icl_results_{ds_name}_{icl_method}_{model_name}.csv', index=False)

Things to do:

Mistral 8B?
Law dataset
Figure out max tokens crap


### Law QA

In [ ]:

max_num_egs = 3

data = load_dataset('dzunggg/legal-qa-v1')['train']
bleu = evaluate.load('bleu')

tokenizer = tokenizer_gpt2
model = model_gpt2

def filter_example(example):
    return count_tokens(tokenizer, f"### Question: {example['question']}\n ### Answer: {example['answer']}") <= 300

data = data.filter(filter_example)

train_test_split = data.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test'].select(range(100))


print("Length of test set: ", len(test_dataset))
train_list = format_examples(train_dataset, ds_name='lawqa')
print("Length of train set", len(train_list))

max_token_dict = {}
for eg in test_dataset:
    max_token_dict[eg['answer']] = count_tokens(tokenizer, eg['answer'])

# avg_tokens = 0
# for eg in train_list:
#     avg_tokens+= count_tokens(tokenizer_mist, eg)
# avg_tokens  = avg_tokens/len(train_list)
# print("AVG TOKENS: ",avg_tokens)
#avg_tokens = 



# print("Token Dict complete")

icl_method = 'random'
model_name = 'gpt2'
ds_name = 'lawqa'

bert_scores = []
results_data = []


for i in range(max_num_egs):
    accuracy = 0
    reals, preds = evaluate_icl(train_list, test_dataset, model, tokenizer, i, model_name=model_name, ds_name=ds_name, method=icl_method, max_tokens_dict=max_token_dict)
    P, R, F1 = bert_score.score(preds, reals, lang="en")
    average_F1 = sum(F1) / len(F1)
    bert_scores.append(average_F1)
    refs = [[r] for r in reals]
    order = int(sum(len(s) for s in refs)/len(refs))
    bleu_score = bleu.compute(predictions=preds, references=refs, max_order=order)
 
    results_data.append({'num_samples' : len(preds), 'num_demonstrations' : i, 'bert_score' : float(average_F1), 'bleu_score' : bleu_score['bleu']})

results_df = pd.DataFrame(results_data)
results_df.to_csv(f'icl_results/icl_results_{ds_name}_{icl_method}_{model_name}.csv', index=False)